In [ ]:
#Caicuccoder
!pip install pandas numpy scikit-learn matplotlib seaborn keras
!pip install tensorflow
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense

# Bước 1: Đọc dữ liệu file csv để xử lí
data = pd.read_csv('AirQualityUCI.csv') 
data.head()
# Hiển thị thông tin tổng quan về dữ liệu
print("Thông tin về dữ liệu:")
print(data.info())  # Hiển thị kiểu dữ liệu và số lượng giá trị không bị thiếu
# Hiển thị số lượng hàng và cột
print("\nSố lượng hàng và cột:")
print(data.shape)  # Số hàng và số cột
# Hiển thị các thống kê cơ bản của dữ liệu
print("\nThống kê mô tả của dữ liệu:")
print(data.describe())  # Hiển thị các thống kê cơ bản như mean, std, min, max
# Hiển thị tên các cột
print("\nTên các cột:")
print(data.columns)
# Bước 2: Khám phá và tiền xử lý dữ liệu
# Kiểm tra các giá trị thiếu
print(data.isnull().sum())

# Chuyển đổi cột Date và Time thành định dạng datetime
data['Datetime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'])

# Loại bỏ các cột không cần thiết
data = data.drop(['Date', 'Time'], axis=1)

# Xử lý các giá trị âm trong các cột như NMHC_GT
data = data[data['NMHC_GT'] >= 0]

# Mã hóa cột CO_level thành số nguyên
le = LabelEncoder()
data['CO_level'] = le.fit_transform(data['CO_level'])

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
features = data.drop(['CO_level', 'Datetime'], axis=1)
features_scaled = scaler.fit_transform(features)
# Bước 3: Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(features_scaled, data['CO_level'], test_size=0.2, random_state=42)
# Bước 4: Huấn luyện mô hình Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

# Đánh giá mô hình Random Forest
print("Random Forest Classification Report")
print(classification_report(y_test, rf_predictions))
print("Confusion Matrix")
print(confusion_matrix(y_test, rf_predictions))
# Bước 5: Huấn luyện mô hình mạng nơ-ron
nn_model = Sequential()
nn_model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
nn_model.add(Dense(16, activation='relu'))
nn_model.add(Dense(1, activation='sigmoid'))
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

nn_model.fit(X_train, y_train, epochs=50, batch_size=10, validation_data=(X_test, y_test))

# Dự đoán và đánh giá mô hình mạng nơ-ron
nn_predictions = (nn_model.predict(X_test) > 0.5).astype("int32")
print("Neural Network Classification Report")
print(classification_report(y_test, nn_predictions))
print("Confusion Matrix")
print(confusion_matrix(y_test, nn_predictions))
# Bước 6: Hiển thị ma trận nhầm lẫn
plt.figure(figsize=(10, 6))
sns.heatmap(confusion_matrix(y_test, rf_predictions), annot=True, fmt="d", cmap="Blues")
plt.title("Ma trận nhầm lẫn Random Forest")
plt.xlabel("Dự đoán")
plt.ylabel("Dòng")
plt.show()
# Bước 7: Nhập dữ liệu mới và đưa ra dự đoán
def predict_new_data(data_input):
    """
    Nhận vào một mảng dữ liệu mới và đưa ra dự đoán của cả mô hình Random Forest và Neural Network.
    """
    # Chuyển đổi dữ liệu đầu vào thành dạng numpy array và chuẩn hóa
    data_input_scaled = scaler.transform([data_input])

    # Dự đoán bằng mô hình Random Forest
    rf_prediction = rf_model.predict(data_input_scaled)[0]
    rf_prediction_label = le.inverse_transform([rf_prediction])[0]

    # Dự đoán bằng mô hình Neural Network
    nn_prediction = (nn_model.predict(data_input_scaled) > 0.5).astype("int32")[0][0]
    nn_prediction_label = le.inverse_transform([nn_prediction])[0]

    print(f"Dự đoán bằng Random Forest: {rf_prediction_label}")
    print(f"Dự đoán bằng Neural Network: {nn_prediction_label}")

# Ví dụ về dữ liệu mới cần dự đoán 
new_data = [150, 136, 2.3, 180, 13.4, 9.0, 0.9, 1200, 34.5, 1.2, 0.7, 300, 27] 

# Dự đoán cho dữ liệu mới
predict_new_data(new_data)

